# Table of Contents

- [Introduction](#section-1)
- [Analysis by author type](#section-2)
    - [Number of mentions of each concept split by Party/non-Party stakeholders](#section-2-2)
    - [Number of documents mentioning each concept split by Party/non-Party stakeholders](#section-2-3)
    - [Number of authors mentioning each concept split by Party/non-Party stakeholders](#section-2-4)
    - [Percentage of mentions of each concept by Party/non-Party stakeholders](#section-2-5)
    - [Percentage of documents mentioning each concept by Party/non-Party stakeholders](#section-2-6)
    - [Percentage of authors mentioning each concept by Party/non-Party stakeholders](#section-2-7)
- [Granular Breakdown of Concepts](#section-3)
    - [Total Documents](#section-3-1)
    - [Total Mentions](#section-3-2)
    - [Total Authors](#section-3-3)
- [Geographical distribution of concepts](#section-4)
- [Sample mentions of concepts](#section-5)
- [Word Co-Occurrences](#section-6)
- [Annex: Methodology](#section-7)
- [Explanation of the linguistic rules using OpenAI GPT-4](#section-8)

In [ ]:
import re
from datetime import datetime
from pathlib import Path

import geopandas as gpd
import numpy as np
import pandas as pd
import pycountry
from IPython.core.display_functions import display
from IPython.display import Markdown
from wbgapi import economy
from wordcloud import STOPWORDS
import altair as alt

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# set the display options to allow resizing columns
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
concepts_path = Path().absolute().parent / 'concepts'

In [ ]:
concept = "fossil-fuels"
date_of_nb = datetime.now().strftime("%d-%m-%Y")
df_concepts = pd.read_excel(concepts_path / concept / "output_with_metadata.xlsx")
df_spans = pd.read_csv(concepts_path / concept / "spans.csv")
date_of_nb = datetime.today().strftime('%d-%m-%Y')

In [ ]:
df_concepts = pd.read_excel(concepts_path / concept / "output_with_metadata.xlsx")
df_spans = pd.read_csv(concepts_path / concept / "spans.csv")

In [ ]:
def find_country(text: str) -> str:
    """
    Given a text string, attempts to find the name of a country
    mentioned in the text. Returns the name of the country if found,
    or None otherwise.
    """
    # Check for uppercase and lowercase country name
    for country in pycountry.countries:
        if country.name.upper() in text.upper():
            return country.name
        elif country.name.lower() in text.lower():
            return country.name

    # Check for common name
    for country in pycountry.countries:
        if country.name in text:
            return country.name
        if hasattr(country, 'common_name') and country.common_name in text:
            return country.common_name

        # Check for official name
        if hasattr(country, 'official_name') and country.official_name in text:
            return country.official_name

        # Check for alpha_2 code (e.g., "US" for United States)
        if country.alpha_2 in re.findall(r'\b[A-Z]{2}\b', text):
            return country.name

        # Check for alpha_3 code (e.g., "USA" for United States)
        if country.alpha_3 in re.findall(r'\b[A-Z]{3}\b', text):
            return country.name

    return None


def get_country_code(x: str) -> str:
    """
    Given the name of a country, returns its ISO 3166-1 alpha-3
    code. Returns None if the country is not found.
    """
    try:
        return pycountry.countries.get(name=x).alpha_3
    except (AttributeError, LookupError):
        return None

def create_docs_table(df_concepts):

    # First, create a new column that categorizes each row as 'Party' or 'Non-Party'
    df_concepts['category'] = np.where(df_concepts['Party'].notna(), 'Party', 'Non-Party')

    # Use groupby to count the unique document_id's for each category and for the total dataset
    docs_analysed = df_concepts.groupby('category')['document_id'].nunique()

    # Convert the resulting Series into a DataFrame with a single row and a custom index
    docs_analysed_table = pd.DataFrame(docs_analysed.values.reshape(1, -1),
                                       columns=docs_analysed.index,
                                       index=["Documents"])

    return docs_analysed_table

start_col_name = "text"
end_col_name = "document_id"
start_col = df_concepts.columns.get_loc(start_col_name)
end_col = df_concepts.columns.get_loc(end_col_name)
indicator_columns = df_concepts.columns[start_col + 1:end_col]
# Melt the DataFrame and specify the columns to keep as id_vars
df_concepts = df_concepts.rename(columns={'party': 'Party'})
df_concepts['category'] = np.where(df_concepts['Party'].notna(), 'Party', 'Non-Party')
df_concepts_melted = df_concepts.melt(id_vars=[col for col in df_concepts.columns if col not in indicator_columns], var_name="Concept", value_name="value")
# filter where indicators are 1
df_concepts_melted = df_concepts_melted[df_concepts_melted["value"] == 1]
# Create a new column 'country' with the found country names
df_concepts_melted["document_name_x_reformatted"] = df_concepts_melted["document_name_x"].str.replace(r'[_20]+', ' ', regex=True)
df_concepts_melted["document_name_y_reformatted"] = df_concepts_melted["document_name_y"].str.replace(r'[_20]+', ' ', regex=True)
df_concepts_melted['country_x'] = df_concepts_melted['document_name_x_reformatted'].apply(find_country)
df_concepts_melted['country_y'] = df_concepts_melted['document_name_y_reformatted'].apply(find_country)
df_concepts_melted['country'] = df_concepts_melted['country_x'].combine_first(df_concepts_melted['country_y'])
# create 3 letter country code
df_concepts_melted['country_code'] = df_concepts_melted['country'].apply(get_country_code)
# create 3 letter country code
df_concepts_melted['country_code'] = df_concepts_melted['country'].apply(get_country_code)
df_eco = pd.DataFrame(economy.list())
# Assuming the 3-letter country code column in df_concepts_melted is named 'country_code'
df_concepts_melted = pd.merge(df_concepts_melted, df_eco[['id', 'region']], left_on='country_code', right_on='id', how='left')

df_documents=df_concepts_melted.groupby(['Concept', 'category'])['document_id'].nunique().reset_index().pivot(index='Concept', columns='category', values='document_id')
df_documents['Total'] = df_documents.sum(axis=1)
df_documents.columns.name = None
df_mentions = df_concepts_melted.groupby('Concept')['category'].value_counts().rename('count').reset_index().pivot(index='Concept', columns='category', values='count')#.reset_index()
df_mentions['Total'] = df_mentions.sum(axis=1)
df_mentions.columns.name = None
df_authors = df_concepts_melted.groupby(['Concept', 'category'])['author'].nunique().reset_index().pivot(index='Concept', columns='category', values='author')
df_authors['Total'] = df_authors.sum(axis=1)
df_authors.columns.name = None

# group melted df by country_code and Concept and number of concept
dd=df_concepts_melted.groupby(['country_code', 'Concept'])['Concept'].count().rename('count').reset_index()
world = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
merged = world.set_index("iso_a3").join(dd.set_index("country_code"),how="inner")
# where concept is nan, create new rows for all concepts with count 0
# merged = merged.reset_index().melt(id_vars=['country', 'geometry'], value_vars=merged.columns[2:], value_name='count').dropna(subset=['count'])

# get a unique mapping between document_id and document_name_y for the df_concepts_melted
mapping = df_concepts_melted[['document_id', 'document_name_y', 'category','author']].drop_duplicates()
# now apply this mapping to df_spans to create a new column with the document_name

df_spans['processed_sentence']=df_spans['sentence'].apply(lambda x: ' '.join([word for word in x.split() if word not in (STOPWORDS)]))
df_spans['normalised_text']=df_spans['text'].str.lower()
df_spans = df_spans.merge(mapping, on=['document_id'], how='left')
merged=merged.reset_index().rename(columns={'index': 'iso_a3'})
# for every country in world, check if the concept is in merged. If not, add a row with count 0
for country in world['iso_a3']:
    for concept in merged['Concept'].unique():
        if not merged[(merged['iso_a3']==country) & (merged['Concept']==concept)].empty:
            continue
        else:
            merged = pd.concat([merged, world[world['iso_a3']==country].reset_index(drop=True).merge(pd.DataFrame({'Concept': concept, 'count': 0}, index=[0]), left_index=True, right_index=True)], ignore_index=True)


merged = merged.rename(columns={'name': 'country'})

<a id='section-1'></a>
# Introduction

In [ ]:
report_text = f"""
 This report contains summary statistics and visualisations for all identified mentions of {concept} within the Global Stocktake submissions, as of {date_of_nb}. It summarises the total number of mentions of {concept} and the number of documents these mentions come from,  broken down by Party and non-Party stakeholders. It also includes a full list of the Parties and non-Party stakeholders which mention {concept} within the Global Stocktake submissions, broken down by subtypes. A sample of the extracts where {concept} is mentioned is included below, with a link to the full set provided. The technical annex contains the methodology used to source the data for this report.
"""
display(Markdown(report_text))

<a id='section-2'></a>
# Analysis by author type

In [ ]:
docs_analysed_table = create_docs_table(df_concepts_melted)
docs_analysed_table_melted = docs_analysed_table.reset_index().melt(id_vars=["index"], var_name="stakeholder", value_name="count")
chart = alt.Chart(docs_analysed_table_melted).mark_bar().encode(
    x='index:N',
    y='count:Q',
    color='stakeholder:N',
    order=alt.Order('stakeholder:N', sort='ascending')
).properties(title=f"Number of documents analysed by author type", width=400, height=300)

chart


<a id='section-2-2'></a>
## Number of mentions of each concept split by Party/non-Party stakeholders

In [ ]:
display(df_mentions)

<a id='section-2-3'></a>
## Number of documents mentioning each concept split by Party and non-Party stakeholders

In [ ]:
display(df_documents)

<a id='section-2-4'></a>
## Number of authors mentioning each concept split by Party/non-Party stakeholders

In [ ]:
display(df_authors)

In [ ]:
def plot_percentages_stacked(df):
    df=df.reset_index()
    df['Non-Party_percentage'] = df['Non-Party'] / df['Total'] * 100
    df['Party_percentage'] = df['Party'] / df['Total'] * 100

    # Melt the DataFrame to have columns: Concept, Group, and Percentage
    df_melted = df.melt(id_vars='Concept', value_vars=['Non-Party', 'Party'], var_name='Group', value_name='Percentage')

    base = alt.Chart(df_melted).encode(
        x=alt.X('Concept:N', title='Concept'),
        y=alt.Y('Percentage:Q', title='Percentage', stack='normalize', axis=alt.Axis(format='%')),
        color=alt.Color('Group:N', title='Group', scale=alt.Scale(scheme='tableau10')),
        tooltip=['Concept', 'Group', 'Percentage']
    )

    # Create the bar chart
    bar_chart = base.mark_bar().encode()

    display(bar_chart)

<a id='section-2-6'></a>
## Percentage of documents mentioning each concept by Party/non-Party stakeholders

In [ ]:
# number of unique documents mentioning each concept
data_docs=(100*df_concepts_melted.groupby(['category','Concept']).document_id.nunique()/df_concepts_melted.groupby('category').document_id.nunique()).rename('Percentage').reset_index()

In [ ]:
alt.Chart(data_docs).mark_bar().encode(
    x=alt.X('category:O'),
    y='Percentage:Q',
    color='category:N',
    column='Concept:N',
)

<a id='section-2-7'></a>
## Percentage of authors mentioning each concept by Party/non-Party stakeholders

In [ ]:
# of all the Party/non-Party-authors in the corpus, what percentage have documents containing mentions of each concept?
data_authors=(100*df_concepts_melted.groupby(['category','Concept']).author.nunique()/df_concepts_melted.groupby('category').author.nunique()).rename('Percentage').reset_index()
# make sure chart has no x-axis labels
chart=alt.Chart(data_authors).mark_bar().encode(
    x=alt.X('category:O', axis=alt.Axis(labels=True)),
    y='Percentage:Q',
    color='category:N',
    column='Concept:N',
)
chart

<a id='section-3'></a>
# Granular Breakdown of Concepts

In [ ]:
# Prepare the data
total_concept_mentions = df_spans.rename(columns={'type':'concept'}).groupby(['id', 'concept']).document_id.count().reset_index().rename(columns={'document_id':'count'})
document_concept_mentions = df_spans.rename(columns={'type':'concept'}).groupby(['id', 'concept']).document_id.nunique().reset_index().rename(columns={'document_id':'count'})
author_concept_mentions = df_spans.rename(columns={'type':'concept'}).groupby(['id', 'concept']).author.nunique().reset_index().rename(columns={'author':'count'})

def plot_concept_mentions(data, title):
    # Create a base chart
    base_chart = alt.Chart(data).mark_bar().encode(
        y=alt.Y('count:Q', title=title),
        x=alt.X('id:N', sort='-y',axis=alt.Axis(title=None, labels=True)),
        color=alt.Color('concept:N', scale=alt.Scale(scheme='category10')),
        tooltip=['count:Q'],
    ).properties(
        width=200,
        height=300,
    )

    # Create faceted chart
    faceted_chart = base_chart.facet(
        facet=alt.Facet('concept:N', title=None),
        columns=2,
        spacing=15,
        title=title
    ).resolve_scale(x='independent')

    display(faceted_chart)

<a id='section-3-1'></a>
## Total Mentions

In [ ]:
plot_concept_mentions(total_concept_mentions, 'Total mentions of each concept')

<a id='section-3-2'></a>
## Total Documents

In [ ]:
plot_concept_mentions(document_concept_mentions, 'Number of documents mentioning each concept')

<a id='section-3-3'></a>
## Total Authors

In [ ]:
plot_concept_mentions(author_concept_mentions, 'Number of authors mentioning each concept')

<a id='section-4'></a>
# Geographical distribution of concepts

In [ ]:
# Function to create a choropleth map for a given concept
def create_choropleth_map(merged, concept):
    choropleth_map = (
        alt.Chart(merged[merged['Concept'] == concept])
        .mark_geoshape(stroke="black", strokeWidth=1)
        .encode(
            color=alt.Color("count:Q", scale=alt.Scale(scheme="viridis")),
            tooltip=["country:N", "count:Q"],
        )
        .properties(width=800, height=400, title=f"Number of Mentions of {concept} by Country")
    )
    return choropleth_map

# Iterate over the unique concepts and create a separate choropleth map for each
unique_concepts = sorted(merged['Concept'].unique())
for concept in unique_concepts:
    choropleth_map = create_choropleth_map(merged, concept)
    choropleth_map.display()

<a id='section-5'></a>
# Sample mentions of concepts

Here is a sample of mentions of concepts in the documents. To see the full list, see here (TODO: add link to external Excel download).

In [ ]:
# get a unique mapping between document_id and document_name_y for the df_concepts_melted
mapping = df_concepts_melted[['document_id', 'document_name_y', 'category']].drop_duplicates()
# now apply this mapping to df_spans to create a new column with the document_name
df_spans = df_spans.merge(mapping, on=['document_id'], how='left')

In [ ]:
# df_spans_sample = df_spans[['document_name_y', 'type','category', 'sentence']].dropna().sample(10)

<a id='section-6'></a>
# Word Co-Ocurrences

In [ ]:
import nltk
from nltk import FreqDist
from nltk.util import ngrams
from nltk.tokenize import word_tokenize

nltk.download('punkt', quiet=True);

In [ ]:
def extract_ngrams(df, concept, n, text_col="processed_sentence"):
    # Filter the DataFrame for the given concept
    concept_df = df[df['type'] == concept]

    # Tokenize the sentences and extract n-grams
    tokens = [word_tokenize(sentence) for sentence in concept_df[text_col]]
    ngram_list = [ngram for sentence in tokens for ngram in ngrams(sentence, n)]

    # Filter out n-grams containing non-word characters
    word_ngrams = [ngram for ngram in ngram_list if all(re.match(r'^\w+$', word) for word in ngram)]

    # Calculate the frequency distribution of n-grams
    freq_dist = FreqDist(word_ngrams)

    return freq_dist



In [ ]:
concepts = df_spans.type.unique()

# Set the number of top bigrams and trigrams to display
num_top_ngrams = 10

for concept in concepts:
    display(Markdown(f"## Top {num_top_ngrams} bigrams and trigrams in sentences relating to {concept.title()}\n"))

    # Extract bigrams and trigrams for the given concept
    bigrams_freq = extract_ngrams(df_spans, concept, n=2, text_col="processed_sentence")
    trigrams_freq = extract_ngrams(df_spans, concept, n=3, text_col="processed_sentence")

    # Create DataFrames for bigrams and trigrams
    bigrams_df = pd.DataFrame(bigrams_freq.most_common(num_top_ngrams), columns=['Bigrams', 'Frequency'])
    trigrams_df = pd.DataFrame(trigrams_freq.most_common(num_top_ngrams), columns=['Trigrams', 'Frequency'])

    display(Markdown(f"### Bigrams\n"))
    display(bigrams_df)
    display(Markdown(f"### Trigrams\n"))
    display(trigrams_df)


In [ ]:
import warnings

warnings.filterwarnings("ignore")

infile=pd.read_excel("/home/stefan/PycharmProjects/global-stocktake/concepts/fossil-fuels/input.xlsx", header=[0,1]);

In [ ]:
excel_string = infile.to_csv(sep='\t', index=False, header=False)

In [ ]:
#, header=[0,1])
# infile.columns = infile.columns.map(' - '.join)
infile.columns = [' - '.join(col).strip() if 'Unnamed' not in col[1] else col[0] for col in infile.columns]

<a id='section-7'></a>
# Annex: Methodology

In [ ]:
display(Markdown(f"""
The documents within the Global Submissions Information Portal were searched for all mentions of {concept}. The documents are all automatically translated into English using a Google Translate API, and the full text of all documents are then searched and return mentions of {concept} within the translated or original text.

To capture the variations of the expression of {concept} within the Global Stocktake submissions, the following input file was created (TODO: Provide link to input file)."""))

display(infile)


display(Markdown(F"""The root forms of the terms were used so that all variations of these listed terms were searched. For example, ‘technology’ would return ‘technologies’ and ‘technological’.

Terms connected with a hyphen are also returned separately. For example, ‘coal-fired’ would return ‘coal-fired’ and ‘coal fired’."""))


In [ ]:
# Note: you need to be using OpenAI Python v0.27.0 for the code below to work
import openai

openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

def explain_excel_content(prompt):
    response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are an academic who provides intuitive explanations of complex concepts."},
        {"role": "user", "content": f"{prompt}"},

    ],
    temperature=0.0,
    )
    return response['choices'][0]['message']['content']

# Describe the content of the Excel file
# Replace this description with the actual content or structure of your Excel file
excel_file = infile


# Generate a prompt for the OpenAI API
prompt = f"Please provide a short intuitive explanation of the spacy linguistic rules in this copy and paste from a excel file. Assume the user doesn't have any linguistic knowledge: {excel_string}"

# Send the request to the OpenAI API
explanation = explain_excel_content(prompt)

<a id='section-8'></a>
# Explanation of the linguistic rules using OpenAI GPT-4

In [ ]:
# display the explanation accounting for \n
display(Markdown(f"{explanation}"))

In [240]:
# display the explanation accounting for \n
display(Markdown(f"{explanation}"))

This text appears to be a list of linguistic rules for categorizing different types of fossil fuels and their related terms. The rules are organized in a table format, with each row representing a specific rule. The first column indicates the general category (e.g., Oil, Gas, Coal, or Fossil fuel), and the subsequent columns provide information on how to identify and classify terms related to that category.

The rules use various linguistic features to classify terms, such as:

1. LOWER: This feature checks if the term is in lowercase (e.g., "oil" or "gas").
2. LEMMA: This feature checks the base form of a word (e.g., "sand" for "sands" or "peatland" for "peatlands").
3. ORTH: This feature checks the spelling of a word (e.g., "-" for "oil-fired" or "coal-water").
4. OP: This feature checks the presence of a specific character (e.g., "?" for "oil-fired" or "coal-water").
5. IS_UPPER: This feature checks if the term is in uppercase (e.g., "LNG" or "LPG").

For example, the rule for "Oil" and "oil" in lowercase (Oil, oil, LOWER, oil) means that if the term "oil" is found in lowercase, it should be classified under the "Oil" category. Similarly, the rule for "Gas" and "LNG" in uppercase (Gas, LNG, LOWER, LNG, IS_UPPER, YES) means that if the term "LNG" is found in uppercase, it should be classified under the "Gas" category.

In summary, this list of linguistic rules helps to categorize and classify terms related to fossil fuels based on their spelling, capitalization, and base forms.